# Goal: Create Interactive Map of 2025 NCAA Participants

In [61]:
### Setup and Dependencies

### Import constants from config.py
from config import *

### Dependencies
import re
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3  # Assuming SQL connection for database operations

### MATPLOTLIB WITH ACCESORIES
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase
from PIL import Image
## Map visualization
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium.features import CustomIcon


### FILE PATHS
TEMP_FOLDER = '../TEMP/'
DATA_FOLDER = '../data/'

## 2024-25 Full Roster Path
roster_path = DATA_FOLDER + 'roster_2025_current_march_25_v4_ex20250325.csv'

full_roster = pd.read_csv(roster_path) # load roster as dataframe
full_roster.info() # Check to make sure it loaded correctly



### SCHOOL INFO TABLE FOR LOGO PATHS
school_info_path = os.path.join('..', 'data', 'arena_school_info.csv')
school_info_df = pd.read_csv(school_info_path) # Load school info

# Path to logo folder
logo_folder = os.path.join('..', 'images', 'logos')

### SHAPEFILES
# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'combined-us-canada.geojson')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)

## CHECK SHAPEFILES FOR COMPATIBILITY
# Set the CRS for both dataframes if it's missing
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

if gdf_states.crs is None:
    gdf_states.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Current Team    1836 non-null   object 
 1   Last_Name       1836 non-null   object 
 2   First_Name      1836 non-null   object 
 3   No              1836 non-null   int64  
 4   Position        1836 non-null   object 
 5   Yr              1836 non-null   object 
 6   Ht              1836 non-null   object 
 7   Wt              1836 non-null   int64  
 8   DOB             1835 non-null   object 
 9   Hometown        1835 non-null   object 
 10  Height_Inches   1836 non-null   int64  
 11  Draft_Year      228 non-null    float64
 12  NHL_Team        228 non-null    object 
 13  D_Round         228 non-null    float64
 14  Last Team       1828 non-null   object 
 15  League          1781 non-null   object 
 16  City            1835 non-null   object 
 17  State_Province  1835 non-null   o

#### Roster Hotfix
- replace missed location abbreviations with full names
    - RUS = Russia
    - IN = S_P 'Indiana' + Country 'USA'
    - UKR = Ukraine
    - JPN = Japan

In [62]:
## Location abbreviation replacement dictionary
loc_replace_dict = {'RUS':'Russia', 'IN':'Indiana', 'UKR':'Ukraine', 'JPN':'Japan'}

# Check the State_Province column and replace the abbreviations with full names
full_roster['State_Province'] = full_roster['State_Province'].replace(loc_replace_dict)
# same with the Country column
full_roster['Country'] = full_roster['Country'].replace(loc_replace_dict)

# If Country = Indiana, change to USA
full_roster['Country'] = full_roster['Country'].replace('Indiana', 'USA')

In [63]:
### Filter Roster to only teams in the NCAA Tournament
print(ncaa_team_list_2025) # Check the list of Tourney teams from the config file

############ TURNED OFF FOR NOW
# Filter to only teams in the NCAA Tournament
# roster_ncaa = full_roster
roster_ncaa = full_roster[full_roster['Current Team'].isin(ncaa_team_list_2025)]

roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency

# Create a new 'Player' column that combines the player's first and last name
roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']

roster_ncaa['Player'] = roster_ncaa['Player'].str.strip() # Strip any leading or trailing white space

# roster_ncaa.info() # Check to make sure it loaded correctly

['Michigan State', 'Cornell', 'Boston University', 'Ohio State', 'Western Michigan', 'Minnesota State', 'Minnesota', 'Massachusetts', 'Boston College', 'Bentley', 'Providence', 'Denver', 'Maine', 'Penn State', 'Connecticut', 'Quinnipiac']


C:\Users\jbanc\AppData\Local\Temp\ipykernel_27920\2879803183.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa.rename(columns={'Current Team':'Team'}, inplace=True) # Rename 'Current_Team' to 'Team' for consistency
C:\Users\jbanc\AppData\Local\Temp\ipykernel_27920\2879803183.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roster_ncaa['Player'] = roster_ncaa['First_Name'] + ' ' + roster_ncaa['Last_Name']
C:\Users\jbanc\AppData\Local\Temp\ipykernel_27920\2879803183.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

## Add Current Season Stats to Roster

In [64]:
### Load the player_ytd stats table from the database

# connect to the database
conn = sqlite3.connect(recent_clean_db)
# Check the table name in db
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


# Load the player_ytd stats table from the database
player_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd", conn)
player_ytd.rename(columns={'Clean_Player':'Player'}, inplace=True) # Rename 'Clean Player' to 'Player' for consistency
player_ytd['Player'] = player_ytd['Player'].str.strip() # Strip leading and trailing whitespace from the 'Player' column
# player_ytd.info() # Check to make sure it loaded correctly

# Close the connection
conn.close()

### Merge the player_ytd stats table with the roster_ncaa table on Player and Team columns
# roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on=['Player', 'Team'], how='left')
roster_ncaa_ytd = pd.merge(roster_ncaa, player_ytd, on='Player', how='left')

roster_ncaa_ytd.rename(columns={'Team_x':'Team'}, inplace=True)# Rename Team_x back to Team
roster_ncaa_ytd.rename(columns={'Team_y':'Team_from_db'}, inplace=True) # Rename Team_y to Team_from_db

# roster_ncaa_ytd.info() # Check to make sure it merged correctly
# roster_ncaa_ytd.head() # Head of the merged table

[('player_stats_ytd',), ('master_roster',), ('advanced_metrics',), ('game_details',), ('goalie_stats',), ('line_chart',), ('linescore',), ('penalty_summary',), ('player_stats',), ('scoring_summary',)]


#### Filter out Skaters who have not appeared in a game this season

In [65]:
## How many players show no games played?
# no_games = roster_ncaa_ytd[roster_ncaa_ytd['Games_Played'].isnull()]
# print(len(no_games))
# print(no_games)



In [66]:
## Drop any players who have not appeared in a game this season
## Print pre filter length
print(f'PreFiltered Roster Length: {len(roster_ncaa_ytd)}')
roster_ncaa_ytd = roster_ncaa_ytd.dropna(subset=['Games_Played'])
print(f'PostFiltered Roster Length: {len(roster_ncaa_ytd)}')

### Value count by Position
# print(roster_ncaa_ytd['Position'].value_counts())

PreFiltered Roster Length: 443
PostFiltered Roster Length: 413


In [67]:
# Group by the location columns and count the number of players in each group
location_summary = roster_ncaa_ytd.groupby(['City', 'State_Province', 'Country']).size().reset_index(name='Player_Count')

# Display the resulting DataFrame
print(location_summary)

               City    State_Province   Country  Player_Count
0        Abbotsford  British Columbia    Canada             1
1          Aberdeen          Scotland  Scotland             1
2          Abington     Massachusetts       USA             1
3           Airdrie           Alberta    Canada             1
4        Albert Lea         Minnesota       USA             1
..              ...               ...       ...           ...
309        Woodbury         Minnesota       USA             3
310  Woodcliff Lake        New Jersey       USA             1
311       Woodhaven          Michigan       USA             1
312       Yaroslavl            Russia    Russia             1
313          Zilina          Slovakia  Slovakia             1

[314 rows x 4 columns]


In [68]:
## Value COunts of the location data (City, State, Country)
# print(len(roster_ncaa_ytd['City'].value_counts()))
# roster_ncaa_ytd['City'].value_counts()
# roster_ncaa_ytd['State_Province'].value_counts()
# roster_ncaa_ytd['Country'].value_counts()

## Get Geocode Locations for all hometowns in the dataset
- Using Google Maps API

In [69]:
### GEOCODING USING GOOGLE MAPS API 
# LCHECK FOR AND LOAD GEOCODED DATA BEFORE RUNNING - THIS COSTS MONEY

import googlemaps
import pandas as pd
import config




# Initialize the Google Places API client
gmaps = googlemaps.Client(key=config.g_key)

def geocode_google_places(row):
    try:
        location_str = f"{row['City']}, {row['State_Province']}, {row['Country']}"
        print(f"Querying location: {location_str}")  # Debugging output
        geocode_result = gmaps.geocode(location_str)
        
        # Check the API response
        print(f"Geocode result: {geocode_result}")  # Debugging output
        
        # Check if we got a valid result
        if geocode_result and 'geometry' in geocode_result[0]:
            location = geocode_result[0]['geometry']['location']
            return pd.Series([location['lat'], location['lng']])
        else:
            return pd.Series([None, None])  # Return None if no valid result
    except Exception as e:
        print(f"Error encountered: {e}")  # Debugging output
        return pd.Series([None, None])  # Handle errors gracefully

# Apply the geocode function to the data using Google Places API
location_summary[['Latitude', 'Longitude']] = location_summary.apply(geocode_google_places, axis=1)

# Filter out rows with missing coordinates if needed
location_summary_transformed = location_summary.dropna(subset=['Latitude', 'Longitude'])

# Display the cleaned data with coordinates
location_summary_transformed.head()

Querying location: Abbotsford, British Columbia, Canada
Geocode result: [{'address_components': [{'long_name': 'Abbotsford', 'short_name': 'Abbotsford', 'types': ['locality', 'political']}, {'long_name': 'Fraser Valley', 'short_name': 'Fraser Valley', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'British Columbia', 'short_name': 'BC', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Canada', 'short_name': 'CA', 'types': ['country', 'political']}], 'formatted_address': 'Abbotsford, BC, Canada', 'geometry': {'bounds': {'northeast': {'lat': 49.1718, 'lng': -122.072512}, 'southwest': {'lat': 49.002291, 'lng': -122.462825}}, 'location': {'lat': 49.0504377, 'lng': -122.3044697}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 49.1718, 'lng': -122.072512}, 'southwest': {'lat': 49.002291, 'lng': -122.462825}}}, 'place_id': 'ChIJg7dqcMY1hFQRYAV7KhU1AQU', 'types': ['locality', 'political']}]
Querying location: Aberdeen, Scotland, S

,City,State_Province,Country,Player_Count,Latitude,Longitude
0,Abbotsford,British Columbia,Canada,1,49.050438,-122.304470
1,Aberdeen,Scotland,Scotland,1,57.149889,-2.093753
2,Abington,Massachusetts,USA,1,42.104823,-70.945322
3,Airdrie,Alberta,Canada,1,51.292697,-114.013411
4,Albert Lea,Minnesota,USA,1,43.647801,-93.368656


In [70]:
## Save the geocoded data to a CSV file to avoid re-running the API
location_summary_transformed.to_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv', index=False)


In [71]:
## Load the geocoded data from the CSV file
location_summary_geocoded = pd.read_csv(DATA_FOLDER + '2025_tourney_location_summary_geocoded.csv')

location_summary_geocoded.head()

,City,State_Province,Country,Player_Count,Latitude,Longitude
0,Abbotsford,British Columbia,Canada,1,49.050438,-122.304470
1,Aberdeen,Scotland,Scotland,1,57.149889,-2.093753
2,Abington,Massachusetts,USA,1,42.104823,-70.945322
3,Airdrie,Alberta,Canada,1,51.292697,-114.013411
4,Albert Lea,Minnesota,USA,1,43.647801,-93.368656


In [72]:
### MERGE THE LATITUDE AND LONGITUDE DATA WITH THE ROSTER DATA
roster_ncaa_ytd_location = pd.merge(roster_ncaa_ytd, location_summary_geocoded, on=['City', 'State_Province', 'Country'], how='left')

### HOTFIX FOR Czech Republic ISSUE - NEED TO REPLACE WITH CZECHIA
roster_ncaa_ytd_location['Country'] = roster_ncaa_ytd_location['Country'].replace('Czech Republic', 'Czechia')
# In State_Province column, replace 'Czech Republic' with 'Czechia'
roster_ncaa_ytd_location['State_Province'] = roster_ncaa_ytd_location['State_Province'].replace('Czech Republic', 'Czechia')

# Change Scotland to United Kingdom
roster_ncaa_ytd_location['Country'] = roster_ncaa_ytd_location['Country'].replace('Scotland', 'United Kingdom')
roster_ncaa_ytd_location['State_Province'] = roster_ncaa_ytd_location['State_Province'].replace('Scotland', 'United Kingdom')


# Check the merged data
# print(roster_ncaa_ytd_location.info())
# roster_ncaa_ytd_location.head()



## Talley the Stats by State (Total Games_Played, Goals, Assist, PIM, ect)

In [73]:
# Group by State_Province and add all up player stats to get a state by state summary
state_summary = roster_ncaa_ytd_location.groupby('State_Province').agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'TOI_sec': 'sum',
    'PIM': 'sum',
    'Games_Played': 'sum'
}).reset_index()

# Calculate the average stats per game for each state
state_summary['GPG'] = state_summary['G'] / state_summary['Games_Played']
state_summary['APG'] = state_summary['A'] / state_summary['Games_Played']
state_summary['PtsPG'] = state_summary['Pts'] / state_summary['Games_Played']
state_summary['plus_minus_PG'] = state_summary['plus_minus'] / state_summary['Games_Played']
state_summary['ShPG'] = state_summary['Sh'] / state_summary['Games_Played']
state_summary['TOI_secPG'] = state_summary['TOI_sec'] / state_summary['Games_Played']
state_summary['PIMPG'] = state_summary['PIM'] / state_summary['Games_Played']

# Calculate the number of players per state
state_summary['Player_Count'] = roster_ncaa_ytd_location.groupby('State_Province').size().values
# Calculate average stats per player for each state
state_summary['Games_PlayedPP'] = state_summary['Games_Played'] / state_summary['Player_Count']
state_summary['GPP'] = state_summary['G'] / state_summary['Player_Count']
state_summary['APP'] = state_summary['A'] / state_summary['Player_Count']
state_summary['PtsPP'] = state_summary['Pts'] / state_summary['Player_Count']
state_summary['plus_minus_PP'] = state_summary['plus_minus'] / state_summary['Player_Count']
state_summary['ShPP'] = state_summary['Sh'] / state_summary['Player_Count']
state_summary['TOI_secPP'] = state_summary['TOI_sec'] / state_summary['Player_Count']
state_summary['PIMPP'] = state_summary['PIM'] / state_summary['Player_Count']


# Display the resulting DataFrame
state_summary.head()

,State_Province,G,A,Pts,plus_minus,Sh,TOI_sec,PIM,Games_Played,GPG,...,PIMPG,Player_Count,Games_PlayedPP,GPP,APP,PtsPP,plus_minus_PP,ShPP,TOI_secPP,PIMPP
0,Alaska,14.0,19.0,33.0,16.0,122.0,59596.0,25.0,68.0,0.205882,...,0.367647,2,34.000000,7.00000,9.500000,16.500000,8.000000,61.000000,29798.00000,12.500000
1,Alberta,127.0,177.0,304.0,156.0,972.0,551595.0,233.0,637.0,0.199372,...,0.365777,20,31.850000,6.35000,8.850000,15.200000,7.800000,48.600000,27579.75000,11.650000
2,Arizona,5.0,6.0,11.0,6.0,35.0,20389.0,8.0,25.0,0.200000,...,0.320000,2,12.500000,2.50000,3.000000,5.500000,3.000000,17.500000,10194.50000,4.000000
3,British Columbia,94.0,216.0,310.0,160.0,989.0,587857.0,317.0,695.0,0.135252,...,0.456115,25,27.800000,3.76000,8.640000,12.400000,6.400000,39.560000,23514.28000,12.680000
4,California,132.0,171.0,303.0,159.0,954.0,495127.0,399.0,614.0,0.214984,...,0.649837,23,26.695652,5.73913,7.434783,13.173913,6.913043,41.478261,21527.26087,17.347826


In [74]:
## Export to csv to check in excel
state_summary.to_csv(TEMP_FOLDER + '2025_tourney_state_summary.csv', index=False)

## Output Breakdown Tables: Hometown and First Name

### Breakdown by Hometown

In [75]:
# # ---- 1) GROUP BY CITY, STATE/PROVINCE, COUNTRY ----
# grouped_city = (
#     roster_ncaa_ytd_location
#     .groupby(["City", "State_Province", "Country"], dropna=False)
#     .agg(
#         Count=("Player", "size"),  # number of players from that city
#         Teams=("Team", lambda x: ", ".join(sorted(set(x))))  # unique teams
#     )
#     .reset_index()
# )

# # Sort by Count descending to see which city has the most players
# grouped_city = grouped_city.sort_values(by="Count", ascending=False)

# # (Optional) Keep the top 10
# top_10_cities = grouped_city.head(10)

# # Print or examine the result
# print(top_10_cities)

### Breakdown by First Name

In [76]:
# # roster_ncaa_ytd_location.info()

# # Strip any leading or trailing whitespace from the 'First_Name' and 'Last_Name' columns
# roster_ncaa_ytd_location['First_Name'] = roster_ncaa_ytd_location['First_Name'].str.strip()
# roster_ncaa_ytd_location['Last_Name'] = roster_ncaa_ytd_location['Last_Name'].str.strip()

# # ---- 2) GROUP BY FIRST NAME ----
# grouped_names = (
#     roster_ncaa_ytd_location
#     .groupby("First_Name", dropna=False)
#     .agg(
#         NumPlayers=("First_Name", "size"),       # total players with that first name
#         NumTeams=("Team", "nunique"),           # how many distinct teams
#         Teams=("Team", lambda x: ", ".join(sorted(set(x)))),
#     )
#     .reset_index()
# )

# # Sort by the number of players descending
# grouped_names = grouped_names.sort_values(by="NumPlayers", ascending=False)

# # (Optional) Take top 10
# top_10_names = grouped_names.head(10)

# # Print or examine the result
# print(top_10_names)

## Create the Map Using Follium

### Get top scorer name and statline for each state

In [77]:
            # Top Scorer: <strong>{row['TopScorer_Name']}</strong><br>
            # {row['TopScorer_StatLine']}

In [78]:
# TopScorer_Name - column name for the top scorer's name
# TopScorer_StatLine - column name for the top scorer's stat line
#### statline will be string such as "20 G, 30 A, 50 Pts, +10, 100 SOG"

roster_df = roster_ncaa_ytd_location
    
# Append a suffix to create a unique key for NA states/provinces.
gdf_states['State_Province'] = gdf_states['name'] + " (NA)"
na_states = set(gdf_states['State_Province'])

# 2. Update the State Province Names to work with something down the line
def update_state_name(x):
    # If the NA version exists, use it; otherwise, leave unchanged.
    if (x + " (NA)") in na_states:
        return x + " (NA)"
    else:
        return x

# Update the State_Province column with the modified names
roster_df["State_Province_Mod"] = roster_df["State_Province"].apply(update_state_name)
# Copy the State_Province_Mod column back to the State_Province column ### ATTEMPTED HOTFIX
roster_df["State_Province"] = roster_df["State_Province_Mod"]

idx = roster_df.groupby("State_Province_Mod")["Pts"].idxmax()

# Use those indices to create a DataFrame of top scoring players per territory.
top_scorers = roster_df.loc[idx].copy()

# Helper function to format the PlusMinus stat with a plus sign if positive.
def format_plusminus(pm):
    try:
        return f"+{pm}" if pm > 0 else f"{pm}"
    except Exception:
        return str(pm)

# Create the TopScorer_StatLine for each player.
top_scorers["TopScorer_StatLine"] = top_scorers.apply(
    lambda row: f"{row['G']} G, {row['A']} A, {row['Pts']} Pts, {format_plusminus(row['plus_minus'])}, {row['Sh']} SOG",
    axis=1
)

# Rename the player's name column to TopScorer_Name.
top_scorers.rename(columns={"Player": "TopScorer_Name"}, inplace=True)

# Create the final DataFrame with the columns you need.
top_scorer_result_df = top_scorers[["State_Province_Mod", "Team", "Hometown", "City", "TopScorer_Name", "TopScorer_StatLine"]].copy()

# Create a new column called 'State_Province' (copy of 'State_Province_Mod') to match the expected output.
top_scorer_result_df['State_Province'] = top_scorer_result_df['State_Province_Mod']

# Optional: Display the first few rows to verify the results.
print(top_scorer_result_df.head())




        State_Province_Mod            Team               Hometown  \
404            Alaska (NA)         Cornell      Anchorage, Alaska   
351           Alberta (NA)      Penn State          Calgary, Alb.   
229           Arizona (NA)  Michigan State      Scottsdale, Ariz.   
18   British Columbia (NA)     Connecticut  North Vancouver, B.C.   
266        California (NA)          Denver      San Diego, Calif.   

                City   TopScorer_Name  \
404        Anchorage    Sullivan Mack   
351          Calgary       Aiden Fink   
229       Scottsdale       Red Savage   
18   North Vancouver  Hudson Schandor   
266        San Diego       Zeev Buium   

                             TopScorer_StatLine         State_Province  
404     8.0 G, 12.0 A, 20.0 Pts, +8.0, 63.0 SOG            Alaska (NA)  
351  23.0 G, 29.0 A, 52.0 Pts, +18.0, 132.0 SOG           Alberta (NA)  
229      5.0 G, 6.0 A, 11.0 Pts, +6.0, 35.0 SOG           Arizona (NA)  
18    10.0 G, 30.0 A, 40.0 Pts, +20.0, 66.0 SO

### Circular Offset helper Function

In [79]:
import math

# Function to apply a circular offset to markers with the same location
def add_circular_offset(lat, lon, count, index, radius=0.007):
    """
    Distributes markers in a circular pattern around a central point.
    The radius increases with the number of markers to prevent overlap.
    
    Modified to produce a larger spread:
    Instead of scaling the radius with (1 + count/4),
    we now scale it with (1 + count/2) for a wider spread.
    """
    # Calculate angle for the current marker (in degrees and then convert to radians)
    angle = (360 / count) * index
    radians = math.radians(angle)
    
    # Increase the radius scaling factor to spread markers further apart.
    dynamic_radius = radius * (1 + (count / 2))
    
    # Calculate offsets for latitude and longitude using circular placement
    lat_offset = lat + (dynamic_radius * math.cos(radians))
    lon_offset = lon + (dynamic_radius * math.sin(radians))
    
    return lat_offset, lon_offset



In [80]:
import os
import math
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap, MarkerCluster
from folium.features import CustomIcon
from markupsafe import Markup

######### RENAME THE DATA TO MERGED_DF
merged_df = roster_ncaa_ytd_location.copy()

# Assign unique index per player in each city group
merged_df['city_group_index'] = merged_df.groupby(['City', 'State_Province', 'Country']).cumcount()

# Assign 'Player_Count' per city directly using transform
merged_df['Player_Count'] = merged_df.groupby(['City', 'State_Province', 'Country'])['First_Name'].transform('count')

# Set Logo Size (width, height in pixels)
logo_size = (55, 55)  # Adjust as needed

# Convert number columns to integer type
int_columns = ['No', 'Height_Inches', 'Wt', 'Draft_Year', 'D_Round', 
               'G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM', 'Games_Played']
for col in int_columns:
    merged_df[col] = merged_df[col].astype('Int64')


def create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states, 
                               map_center=[45.0, -93.0], zoom_start=4):
    """
    Build an interactive map with multiple layers:
      - Base tile layers (dark/light themes)
      - Choropleth shading of territories (with custom bins)
      - Dynamic label markers showing player counts per territory
      - A GeoJSON layer with rich HTML tooltips (season stats + top scorer) on hover
      - Heatmap and marker clusters for individual player markers
      - Custom CSS (including Exo 2 font) for styling
    """
    # -------------------------------
    # 1. Initialize the base map and add tile layers
    # -------------------------------
    folium_map = folium.Map(location=map_center, zoom_start=zoom_start, tiles='OpenStreetMap', name='Default Map')
    folium.TileLayer('CartoDB positron', name='Light Theme', attr=".").add_to(folium_map)
    folium.TileLayer('CartoDB dark_matter', name='Dark Theme', attr=".").add_to(folium_map)
    
    
    # Inject custom CSS to load the Exo 2 font and style tooltips
    font_link = """
    <link href="https://fonts.googleapis.com/css2?family=Exo+2:wght@400;700&display=swap" rel="stylesheet">
    """
    custom_font_css = """
    <style>
        .leaflet-tooltip, .folium-tooltip { 
            font-family: 'Exo 2', sans-serif; 
            font-size: 12px; 
        }
    </style>
    """
    folium_map.get_root().header.add_child(folium.Element(font_link + custom_font_css))
    
    # -------------------------------
    # 2. Prepare GeoDataFrames for territories
    # -------------------------------
    # For detailed North American states, append a suffix for disambiguation.
    gdf_states['State_Province'] = gdf_states['name'] + " (NA)"
    na_states = set(gdf_states['State_Province'])

    # Update merged_df so that NA territories use the disambiguated key.
    def update_state_name(x):
        return x + " (NA)" if (x + " (NA)") in na_states else x
    merged_df['State_Province_Mod'] = merged_df['State_Province'].apply(update_state_name)

    # Load global (country-level) boundaries
    gdf_global = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    # Exclude North American countries for which we have detailed data (USA & Canada)
    na_country_names = ['United States of America', 'Canada']
    gdf_global_non_na = gdf_global[~gdf_global['name'].isin(na_country_names)].copy()
    gdf_global_non_na['State_Province'] = gdf_global_non_na['name']

    # Combine detailed NA data with global data
    gdf_combined = pd.concat([gdf_states, gdf_global_non_na], ignore_index=True)

    # -------------------------------
    # 3. Build the choropleth layer based on player counts
    # -------------------------------
    territory_counts = merged_df['State_Province_Mod'].value_counts()
    territory_counts_df = pd.DataFrame(territory_counts).reset_index()
    territory_counts_df.columns = ['State_Province', 'Player_Count']

    #### ORIGINAL BINS FOR NCAA TOURNAMENT MAP
    custom_bins = [0, 1, 5, 10, 20, 30, 40, 50, 60]
    #### Larger bin for when using full roster maxx of 250
    # custom_bins = [0, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 250]
    geojson_data = gdf_combined.__geo_interface__

    folium.Choropleth(
        geo_data=geojson_data,
        data=territory_counts_df,
        columns=['State_Province', 'Player_Count'],
        key_on='feature.properties.State_Province',
        fill_color='YlGn',
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name='Number of Players by Territory',
        bins=custom_bins,
        reset=True,
        name='Shade by Player Count'
    ).add_to(folium_map)

    # -------------------------------
    # 4. Add dynamic label markers showing player counts
    # -------------------------------
    gdf_combined_subset = gdf_combined[['State_Province', 'geometry']]
    territory_counts_gdf = gdf_combined_subset.merge(territory_counts_df, on='State_Province', how='left')
    territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid

    labels_layer = folium.FeatureGroup(name='Player Count by Territory')
    for idx, row in territory_counts_gdf.iterrows():
        if pd.notnull(row['Player_Count']):
            lat, lon = row['centroid'].y, row['centroid'].x
            player_count = int(row['Player_Count'])
            label = folium.Marker(
                location=[lat, lon],
                icon=folium.DivIcon(
                    html=f'''
                        <div class="dynamic-label" style="
                            font-family: 'Exo 2', sans-serif;
                            font-weight: bold;
                            font-size: 16px;
                            color: black;
                            text-align: center;
                            padding: 2px;
                        ">
                            {player_count}
                        </div>
                    '''
                )
            )
            labels_layer.add_child(label)
    labels_layer.add_to(folium_map)

    # Add custom JavaScript for dynamic font sizing on zoom
    dynamic_font_script = """
    <script>
        function updateLabelSizes() {
            var zoom = map.getZoom();
            var newFontSize = Math.max(12, Math.min(30, zoom * 2));
            var labels = document.getElementsByClassName('dynamic-label');
            for (var i = 0; i < labels.length; i++) {
                labels[i].style.fontSize = newFontSize + 'px';
            }
        }
        map.on('zoomend', updateLabelSizes);
        updateLabelSizes();
    </script>
    """
    folium_map.get_root().html.add_child(folium.Element(dynamic_font_script))

    # -------------------------------
    # 5. Merge summary stats and top scorer info for tooltips
    # -------------------------------
    # Update the keys in state_summary for NA territories (append " (NA)" when needed)
    na_original_names = set(gdf_states['name'])
    def add_na_suffix(x):
        return x + " (NA)" if x in na_original_names else x
    state_summary['State_Province'] = state_summary['State_Province'].apply(add_na_suffix)

    # Merge the summary stats and top scorer info with the combined GeoDataFrame
    gdf_tooltip = gdf_combined.merge(state_summary, on="State_Province", how="left")
    gdf_tooltip = gdf_tooltip.merge(top_scorer_result_df, on="State_Province", how="left")

    # Convert TOI_sec to a readable format (HH:MM:SS)
    gdf_tooltip['TOI_sec'] = pd.to_datetime(gdf_tooltip['TOI_sec'], unit='s').dt.strftime('%H:%M:%S')
    gdf_tooltip['TOI_timedelta'] = pd.to_timedelta(gdf_tooltip['TOI_sec'], unit='s')

    # -------------------------------
    # 6. Build custom HTML tooltip content
    # -------------------------------
    def build_tooltip_html(row):
        # Define key fields that must contain data for a tooltip to be shown.
        keys_to_check = ['Player_Count', 'Games_Played', 'TOI_sec', 'G', 'A', 'Pts', 
                        'GPP', 'GPG', 'APP', 'APG', 'PtsPP', 'PtsPG']
        # If all of these are NaN (or missing), return an empty string.
        if all(pd.isnull(row.get(key)) for key in keys_to_check):
            return ""
        
        # Use conditional expressions to avoid converting NaN to integer.
        total_players = int(row['Player_Count']) if pd.notnull(row['Player_Count']) else None
        total_games = int(row['Games_Played']) if pd.notnull(row['Games_Played']) else None
        toi = row.get('TOI_sec')
        
        total_players_str = str(total_players) if total_players is not None else "N/A"
        total_games_str = str(total_games) if total_games is not None else "N/A"
        # Create Time on Ice String in the Format HH:MM:SS (Hour can go above 24) if toi is not None
        toi_str = row['TOI_formatted'] if toi is not None else "N/A"
        
        # Format the top scorer stat line to remove unnecessary decimals (e.g., 20.0 -> 20)
        top_scorer_line = re.sub(r'(\d+)\.0\b', r'\1', str(row['TopScorer_StatLine']))
        
        # Build the tooltip HTML.
        # Note: The header line (with totals) is now placed below the territory name.
        html = f"""
        <div style="font-family: 'Exo 2', sans-serif; font-size: 12px; background-color: white; padding: 5px;">
            <h4 style="margin: 0; text-align: center;">{row['State_Province']}</h4>
            <p style="text-align: center; margin: 5px 0; font-size: 12px;">
                <strong>Total Players:</strong> {total_players_str} &nbsp;&nbsp;
                <strong>Total Games Played:</strong> {total_games_str} &nbsp;&nbsp;<br>
                <strong>Total Time On Ice:</strong> {toi_str}
            </p>
            <table style="width: 100%; border-collapse: collapse; margin-top: 5px;">
                <thead>
                    <tr>
                        <th style="border: 1px solid #ddd; padding: 4px;">Stat</th>
                        <th style="border: 1px solid #ddd; padding: 4px;">Total</th>
                        <th style="border: 1px solid #ddd; padding: 4px;">Per Player</th>
                        <th style="border: 1px solid #ddd; padding: 4px;">Per Game</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td style="border: 1px solid #ddd; padding: 4px;">Goals</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['G']:.0f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['GPP']:.2f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['GPG']:.3f}</td>
                    </tr>
                    <tr>
                        <td style="border: 1px solid #ddd; padding: 4px;">Assists</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['A']:.0f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['APP']:.2f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['APG']:.3f}</td>
                    </tr>
                    <tr>
                        <td style="border: 1px solid #ddd; padding: 4px;">Points</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['Pts']:.0f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['PtsPP']:.2f}</td>
                        <td style="border: 1px solid #ddd; padding: 4px;">{row['PtsPG']:.3f}</td>
                    </tr>
                </tbody>
            </table>
            <p style="margin: 5px 0; text-align: center; font-size: 12px;">
                Top Scorer: <strong>{row['TopScorer_Name']}</strong><br>{row['Hometown']} - <i>{row['Team']}</i><br>
                {top_scorer_line}
            </p>
        </div>
        """
        return html


    # Create a new column in gdf_tooltip with the custom HTML.
    gdf_tooltip['tooltip_html'] = gdf_tooltip.apply(build_tooltip_html, axis=1)
    # Mark the HTML as safe so it isn’t escaped.
    gdf_tooltip['tooltip_html'] = gdf_tooltip['tooltip_html'].apply(Markup)

    # -------------------------------
    # 7. Create GeoJSON layer with rich HTML popups (trigger on click instead of hover)
    # -------------------------------
    # First, filter out features that do not have any tooltip_html content:
    gdf_tooltip_filtered = gdf_tooltip[gdf_tooltip['tooltip_html'].notnull()].copy()
    
    # Now, use GeoJsonPopup to bind the rich HTML content.
    # (GeoJsonPopup shows its content on click instead of on hover.)
    popup = folium.GeoJsonPopup(
        fields=["tooltip_html"],
        aliases=[""],          # no alias label
        localize=True,
        parse_html=True,
        max_width=300,
        sticky=False           # not sticky, so it closes when clicking elsewhere
    )
    
    # Add the GeoJSON layer with the popup to a FeatureGroup.
    popup_layer = folium.FeatureGroup(name="Show Detailed Popups (Click)", show=True)
    folium.GeoJson(
        gdf_tooltip_filtered.__geo_interface__,
        style_function=lambda feature: {
            'fillColor': 'transparent',
            'color': 'transparent',
            'weight': 0
        },
        popup=popup
    ).add_to(popup_layer)
    popup_layer.add_to(folium_map)
    
    # Optionally, add a click handler to the entire map so that clicking anywhere closes any open popup.
    close_on_click_js = f"""
    <script>
        var mapInstance = {folium_map.get_name()};
        mapInstance.on('click', function(e) {{
            mapInstance.closePopup();
        }});
    </script>
    """
    folium_map.get_root().html.add_child(folium.Element(close_on_click_js))

 
    # -------------------------------
    # 8. Add a heatmap layer from merged_df coordinates
    # -------------------------------
    heat_data = [[row['Latitude'], row['Longitude']] for idx, row in merged_df.iterrows()]
    heatmap_layer = folium.FeatureGroup(name='Heatmap', show=True)
    HeatMap(heat_data, radius=25, blur=15, max_intensity=20).add_to(heatmap_layer)
    heatmap_layer.add_to(folium_map)

    # -------------------------------
    # 9. Add marker cluster layer for individual players with custom logo icons
    # -------------------------------
    cluster_group = folium.FeatureGroup(name='Individual Players', control=True, show=False)
    marker_cluster = MarkerCluster(
        spiderfy_on_max_zoom=True,
        show_coverage_on_hover=False,
        max_cluster_radius=20,
        disableClusteringAtZoom=14,
        animateAddingMarkers=True,
        zoomToBoundsOnClick=True
    ).add_to(cluster_group)

    # Center the map on the mean coordinates
    Latitude = merged_df['Latitude'].mean()
    Longitude = merged_df['Longitude'].mean()
    map_instance = folium.Map(location=[Latitude, Longitude], zoom_start=12)
    map_instance.add_child(cluster_group)

    # Toggle the cluster layer based on zoom level (custom JS)
    cluster_toggle_js = f"""
    <script>
        var clusterLayer = {cluster_group.get_name()};
        var map = {map_instance.get_name()};
        map.on('zoomend', function() {{
            if (map.getZoom() >= 14) {{
                if (!map.hasLayer(clusterLayer)) {{
                    map.addLayer(clusterLayer);
                }}
            }} else {{
                if (map.hasLayer(clusterLayer)) {{
                    map.removeLayer(clusterLayer);
                }}
            }}
        }});
    </script>
    """
    map_instance.get_root().html.add_child(folium.Element(cluster_toggle_js))


    # Loop through merged_df to add individual player markers with logos
    for idx, row in merged_df.iterrows():
        team_name = row['Team']
        logo_info = school_info_df[school_info_df['Team'] == team_name]['logo_abv'].values
        if len(logo_info) > 0:
            logo_abv = logo_info[0]
            logo_path = os.path.join(logo_folder, f"{logo_abv}.png")
            if os.path.exists(logo_path):
                logo_icon = CustomIcon(logo_path, icon_size=logo_size)
                player_count = row['Player_Count']
                current_index = row['city_group_index']
                # Compute offsets for overlapping markers
                if player_count > 1:
                    lat_offset, lon_offset = add_circular_offset(row['Latitude'], row['Longitude'], player_count, current_index)
                else:
                    lat_offset, lon_offset = row['Latitude'], row['Longitude']
                
                # Build the player tooltip HTML with consistent styling.
                tooltip_html = f"""
                <div style="font-family: 'Exo 2', sans-serif; font-size: 12px; background-color: white; padding: 5px;">
                    <strong>{row['First_Name']} {row['Last_Name']} - {row['Team']}</strong><br>
                    <span style="color: #333;">{row['Hometown']}</span><br>
                    <span style="color: #333;">{row['Yr']} {row['Position']}</span><br>
                    {"<div style='font-size: 12px; color: gray; margin-top: 5px;'>SEASON STATS:<br>" +
                    f"{row['Games_Played']} GP, {row['G']} G, {row['A']} A, {row['Pts']} PTS, {row['PIM']} PIM</div>" 
                    if pd.notna(row['Games_Played']) else ""}
                </div>
                """
                folium.Marker(
                    location=[lat_offset, lon_offset],
                    tooltip=folium.Tooltip(tooltip_html),
                    icon=logo_icon
                ).add_to(marker_cluster)
    cluster_group.add_to(folium_map)


    # -------------------------------
    # 10. Add layer control and custom CSS for styling
    # -------------------------------
    folium.LayerControl().add_to(folium_map)
    custom_css = """
    <style>
        /* Style the layer control container */
        .leaflet-control-layers {
            background-color: #fff;       /* White background */
            border: 2px solid #ccc;       /* Light gray border */
            border-radius: 5px;
            padding: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.2);
        }
        /* Style the list of layers */
        .leaflet-control-layers-list {
            font-size: 16px;  
            line-height: 1.5;
        }
        /* Style the checkboxes/radio buttons */
        .leaflet-control-layers input[type="radio"],
        .leaflet-control-layers input[type="checkbox"] {
            transform: scale(1.5);
            margin-right: 8px;
        }
        /* Optional: change the color of the layer names */
        .leaflet-control-layers label {
            color: #333;
            font-family: 'Exo 2', sans-serif;
        }
    </style>
    """
    folium_map.get_root().html.add_child(folium.Element(custom_css))
    
    # ### Add behavior to CLOSE TOOLTIP ON CLICK
    close_on_click_js = f"""
    <script>
        var mapInstance = {folium_map.get_name()};
        mapInstance.on('click', function(e) {{
            mapInstance.closeTooltip();
        }});
    </script>
    """
    folium_map.get_root().html.add_child(folium.Element(close_on_click_js))

    # Add a custom reset button in the top right corner
    reset_view_js = f"""
    <script>
        // Get the map instance by its actual name.
        var mapInstance = {folium_map.get_name()};
        // Create a custom control at the top right.
        var resetButton = L.control({{position: 'topright'}});
        resetButton.onAdd = function(map) {{
            var div = L.DomUtil.create('div', 'leaflet-control leaflet-bar leaflet-control-custom');
            div.innerHTML = '<button style="background: white; border: none; padding: 5px 10px; font-family: \\'Exo 2\\', sans-serif; cursor: pointer;">Reset Map</button>';
            // Prevent the click from propagating to the map.
            L.DomEvent.disableClickPropagation(div);
            div.onclick = function() {{
                // Reset to the original center and zoom.
                map.setView([{map_center[0]}, {map_center[1]}], {zoom_start});
            }};
            return div;
        }};
        resetButton.addTo(mapInstance);
    </script>
    """
    folium_map.get_root().html.add_child(folium.Element(reset_view_js))


    ### Add A Note / Attribution box
    attribution_html = """
    <div style="
        position: fixed;
        bottom: 20px;
        right: 20px;
        z-index: 9999;
        background-color: rgba(255, 255, 255, 0.8);
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 5px;
        font-family: 'Exo 2', sans-serif;
        font-size: 14px;
        color: #333;
    ">
        Map created with Folium and OpenStreetMap<br>
        Created by J.Smith @ncaadatasauce.bsky.social<br>
        <span style="font-size: 11px;">Roster and Statistical Data: CollegeHockeyNews.com</span>
    </div>
    """
    folium_map.get_root().html.add_child(folium.Element(attribution_html))



    #### ADD TITLE ANNOTATION BOX
    annotation_html = """
    <div style="
        position: fixed;
        top: 10px;
        left: 50px;
        z-index: 9999;
        background-color: rgba(255, 255, 255, 0.9);
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 5px;
        font-family: 'Exo 2', sans-serif;
        color: #333;
    ">
        <h2 style="margin: 0; font-size: 20px;">The NCAA Hockey Tourney Atlas 2025</h2>
        <p style="margin: 0; font-size: 14px; font-style: italic;">Mapping the Origins of Tournament Competitors</p>
    </div>
    """
    folium_map.get_root().html.add_child(folium.Element(annotation_html))


    return folium_map

# Build the map using your data and then save to an HTML file.
enhanced_player_map = create_map_with_team_logos(merged_df, school_info_df, logo_folder, gdf_states)
enhanced_map_file_path = os.path.join('..', 'TEMP', 'MAP', 'player_origin_map_with_stats_v2.html')


enhanced_player_map.save(enhanced_map_file_path)


C:\Users\jbanc\AppData\Local\Temp\ipykernel_27920\383380329.py:75: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  gdf_global = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\jbanc\AppData\Local\Temp\ipykernel_27920\383380329.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  territory_counts_gdf['centroid'] = territory_counts_gdf.geometry.centroid


ValueError: unit must not be specified if the input contains a str